In [ ]:
import urllib3
import re
from bs4 import BeautifulSoup
import pandas as pd



def main():


    baseurl = 'https://movie.douban.com/top250?start='

    #print(type(baseurl))

    #print(baseurl)
    writetoCSV(baseurl)
    #print(soup)
    # print(soup.title.text)


#def the global variables
#Title
findTitle = re.compile(r'<span class="title">(.*)</span>')
#Score
findScore = re.compile(r'<span class="rating_num" property="v:average">(.*)</span>')
#Number of review
findReview = re.compile(r'<span>(\d*)人评价</span>')
#Brief
findBrief = re.compile(r'<span class="inq">(.*)</span>')
#Background
findBd = re.compile(r'<p class="">(.*?)</p>', re.S)

#爬取网页
def getData(baseurl):
    dataList = []
    for i in range(0, 10):
        http = urllib3.PoolManager()
        url = http.request('GET', baseurl+str(i*25))
        soup = BeautifulSoup(url.data, 'html.parser')
        #print(soup)
        for item in soup.find_all('div', class_="item"):
            # save a movie information
            data = []
            item = str(item)
            # re package: use regular expression to find specific string
            title = re.findall(findTitle, item)
            if (len(title)==2):
                #chinese_title
                title1 = title[0]
                data.append(title1)
                #foreign title
                title2 = title[1].replace("/","")
                #print(title2)
                data.append(title2.strip())
            else: #only one title
                data.append(title[0])
                data.append(' ')

            bd = re.findall(findBd, item)[0]
            bd = re.sub('<br(\s+)?/>(\s+)?'," ",bd)
            bd = re.sub('/'," ",bd)
            bd = bd.strip()
            year = re.findall("\d+",bd)[0]
            #print(bd)
            data.append(year)
            score = re.findall(findScore, item)[0]
            data.append(score)
            review = re.findall(findReview, item)[0]
            data.append(review)
            brief = re.findall(findBrief, item)
            if (len(brief)!=0):
                data.append(brief[0].replace("。",""))
            else:
                data.append(' ')
            #print(data)

            dataList.append(data)

    return dataList

def writetoCSV(baseurl):
    fields = ['Chinese Title', 'Foreign Title', 'Year', 'Score', 'Review', 'Brief']
    rows = getData(baseurl)
    #print(rows)
    # filename = "doubanmovie.csv"
    #
    # with open(filename, 'w') as csvfile:
    #     # creating a csv writer object
    #     csvwriter = csv.writer(csvfile, encoding = 'gbk')
    #
    #     # writing the fields
    #     csvwriter.writerow(fields)
    #
    #     # writing the data rows
    #     csvwriter.writerows(rows)

    test = pd.DataFrame(columns=fields, data=rows)
    test.to_csv('test.csv', encoding='utf-8')

if __name__ == '__main__':
    main()


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

KeyboardInterrupt: ignored

9.7-8.6

In [4]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from googletrans import Translator

In [6]:



def get_detail_page(html):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36"
        }
        cookies = {}
        response = requests.get(url=html, headers=headers, cookies=cookies)
        response.encoding = 'utf-8'
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        print('ERROR')
        time.sleep(3)
        return get_detail_page(html)

def parse_detail_page(data):
    html = get_detail_page(data)
    info = []
    # duration
    try:
        time_pattern = re.compile('property="v:runtime" content="(.*?)"')
        time = re.findall(time_pattern, html)
        info.append(time[0])
    except:
        info.append('1')
    # language
    language_pattern = re.compile('pl">语言:</span>(.*?)<br/>')
    language = re.findall(language_pattern, html)
    info.append(language[0].split(' /')[0])


    return info

def get_movies(link):
    headers = {  # request head
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/83.0.4103.116 Safari/537.36 '

    }
    movie_names=[]
    movie_types=[]
    movie_dis=[]
    movie_grade=[]
    movie_addr=[]
    movie_actor=[]
    movie_director=[]
    movie_year=[]
    movie_cent=[]
    movie_duration =[]
    movie_language =[]
    r=requests.get(link,headers=headers,timeout=10)
    soup=BeautifulSoup(r.text,"lxml")
    
    for each in soup.find_all('div',class_='abstract'):
        a=each.text
        #.match any string
        tp = re.search(r'类型: (.*)',a)
        #print(type(tp.group(1)))
        if tp==None:
          movie_types.append(" ")
        else:
          res = translator.translate(tp.group(1),dest='en').text
          movie_types.append(res)
        actor = re.search(r'主演: (.*)',a)
        if actor==None:
            movie_actor.append(" ")
        else:
            movie_actor.append(translator.translate(actor.group(1),dest='en').text)
        director = re.search(r'导演: (.*)',a)
        if director==None:
            movie_director.append(" ")
        else:
            movie_director.append(translator.translate(director.group(1),dest='en').text)
        addr = re.search(r'制片国家/地区: (.*)',a)
        if addr==None:
            movie_addr.append(" ")
        else:
            movie_addr.append(translator.translate(addr.group(1),dest='en').text)
        year=re.search(r'年份: (.*)',a)
        if year==None:
            movie_year.append(" ")
        else:
            year_str=year.group(1)
            sj=int(year_str[:2])+1
            nd=year_str[2]+'0'
            movie_year.append(year_str)
            movie_cent.append(str(sj)+"."+str(nd))
    #search all class=title div
    div_list=soup.find_all('div',class_='title')
    #time_pattern = re.compile('<a href="(.*?)"')
    #time_pattern = re.compile(r'<a href="(.*?)" target="_blank">')
    #href = re.findall(time_pattern,r.text)[0]
    #print(href)
    for each in div_list:
        movie_name=each.a.text.strip() # get name
        my_re = re.compile(r'[A-Za-z]',re.S)
        res = re.findall(my_re,movie_name)
        if len(res):
          movie_names.append(re.sub('[\u4e00-\u9fa5]', '',movie_name))
        else:
          res = translator.translate(movie_name,dest='en').text
          movie_names.append(res)
        time_pattern = re.compile(r'<a href="(.*?)" target="_blank">')
        href = re.findall(time_pattern,str(each))[0]
        time.sleep(0.5)
        b = parse_detail_page(href)
        #print(b)

        movie_duration.append(b[0])
        movie_language.append(translator.translate(b[1],dest='en').text)
        #print(href)
    for each in soup.find_all('div',class_='rating'):
        a=each.text.split('\n') #get review number
        #get the number from string
        x=re.sub("\D","",a[3])
        movie_dis.append(int(x))
        movie_grade.append(float(a[2]))
    return movie_names,movie_types,movie_dis,movie_grade,movie_addr,movie_actor,movie_director,movie_year,movie_cent,movie_duration,movie_language





In [ ]:
translator = Translator()
movies=get_movies("https://www.douban.com/doulist/240962/")
movies_1=pd.DataFrame({'Name':movies[0],'Type':movies[1],'Duration':movies[9],'Year':movies[7],'Director':movies[6],'Actor':movies[5],'Lanuges':movies[10],'Reviews':movies[2],'Rank':movies[3],
                                           'Country':movies[4],'Century':movies[8]})
#movies_1=[]
for i in range(1,24):
    # total 24 pages, 25 per each
    link="https://www.douban.com/doulist/240962/?start="+str(i*25)
    movies=get_movies(link)
    movies_1=movies_1.append(pd.DataFrame({'Name':movies[0],'Type':movies[1],'Duration':movies[9],'Year':movies[7],'Director':movies[6],'Actor':movies[5],'Lanuges':movies[10],'Reviews':movies[2],'Rank':movies[3],
                                           'Country':movies[4],'Century':movies[8]}),ignore_index=True)
all_movies=movies_1
all_movies
all_movies.to_csv('test.csv', encoding='utf-8-sig')

8.5-8.3

In [8]:
translator = Translator()
movies=get_movies("https://www.douban.com/doulist/243559/")
movies_2=pd.DataFrame({'Name':movies[0],'Type':movies[1],'Duration':movies[9],'Year':movies[7],'Director':movies[6],'Actor':movies[5],'Lanuges':movies[10],'Reviews':movies[2],'Rank':movies[3],
                                           'Country':movies[4],'Century':movies[8]})

for i in range(1,22):
    # total 22 pages, 25 per each
    link="https://www.douban.com/doulist/243559/?start="+str(i*25)
    movies=get_movies(link)
    movies_2=movies_2.append(pd.DataFrame({'Name':movies[0],'Type':movies[1],'Duration':movies[9],'Year':movies[7],'Director':movies[6],'Actor':movies[5],'Lanuges':movies[10],'Reviews':movies[2],'Rank':movies[3],
                                           'Country':movies[4],'Century':movies[8]}),ignore_index=True)
all_movies1=movies_2
all_movies1
all_movies1.to_csv('test1.csv', encoding='utf-8-sig')

8.2-8.0

In [9]:
#translator = Translator()
movies=get_movies("https://www.douban.com/doulist/248893/")
movies_3=pd.DataFrame({'Name':movies[0],'Type':movies[1],'Duration':movies[9],'Year':movies[7],'Director':movies[6],'Actor':movies[5],'Lanuges':movies[10],'Reviews':movies[2],'Rank':movies[3],
                                           'Country':movies[4],'Century':movies[8]})

for i in range(1,22):
    # total 22 pages, 25 per each
    link="https://www.douban.com/doulist/248893/?start="+str(i*25)
    movies=get_movies(link)
    movies_3=movies_3.append(pd.DataFrame({'Name':movies[0],'Type':movies[1],'Duration':movies[9],'Year':movies[7],'Director':movies[6],'Actor':movies[5],'Lanuges':movies[10],'Reviews':movies[2],'Rank':movies[3],
                                           'Country':movies[4],'Century':movies[8]}),ignore_index=True)
all_movies2=movies_3
all_movies2
all_movies2.to_csv('test2.csv', encoding='utf-8-sig')

In [ ]:
!pip uninstall googletrans

Found existing installation: googletrans 3.0.0
Uninstalling googletrans-3.0.0:
  Would remove:
    /usr/local/bin/translate
    /usr/local/lib/python3.7/dist-packages/googletrans-3.0.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/googletrans/*
Proceed (y/n)? y
  Successfully uninstalled googletrans-3.0.0


In [3]:
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 42 kB 1.6 MB/s 
     |████████████████████████████████| 1.3 MB 27.9 MB/s 
     |████████████████████████████████| 53 kB 2.7 MB/s 
     |████████████████████████████████| 65 kB 4.1 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=854614c25a8dc9396adcdda5d5d88f183b1b7575fb0a5386e6cd3eae0e2e6155
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans
